In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager

In [44]:
import time
import random

In [46]:
print(plt.style.available)
%matplotlib inline
driver = webdriver.Chrome()

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [48]:
url_esf='https://sh.esf.fang.com/house-a019-b0103'
driver.get(url_esf)

In [50]:
from selenium.webdriver.common.by import By
#从单个房源容器元素中提取数据
def extract_house_data(house_item):
    data = {}
    # 1. 标题/小区名称 (在 h4 标签内)
    try:
        title_element = house_item.find_element(By.TAG_NAME, 'h4')
        data['Title'] = title_element.text.strip()
    except:
        data['Title'] = 'N/A'

    # 2. 基本信息 (户型、面积、朝向、建成年份)
    # <p class="tel_shop"> ，使用 .text 一次性提取，然后进行清洗。
    try:
        info_text = house_item.find_element(By.CSS_SELECTOR, 'p.tel_shop').text
        # 清洗逻辑（用换行符分割，可能要改
        data['Basic_Info_Raw'] = info_text.replace('\n', ' | ')
    except:
        data['Basic_Info_Raw'] = 'N/A'
    
    # 3. 地址 (在 p class="add_shop" 内)
    try:
        address_container = house_item.find_element(By.CSS_SELECTOR, 'p.add_shop')       
        #小区名称 ：定位 p.add_shop下的第一个a标签
        community_element = address_container.find_element(By.TAG_NAME, 'a')
        data['Community_Name'] = community_element.text.strip()        
        # 提取具体地址：定位 p.add_shop 下的 span 标签
        address_element = address_container.find_element(By.TAG_NAME, 'span')
        data['Detail_Address'] = address_element.text.strip()        
    except:
        data['Community_Name'] = 'N/A'
        data['Detail_Address'] = 'N/A'
        
    # 4. 价格 (在dd class="price_right内)
    try:
        price_dd = house_item.find_element(By.CSS_SELECTOR, 'dd.price_right')
        total_price_element = price_dd.find_element(By.CSS_SELECTOR, 'span.red')
        total_price_raw = total_price_element.text
        data['Total_Price'] = total_price_raw.replace('\n', '').strip() 
        
        unit_price_element = price_dd.find_element(By.XPATH, './span[2]')
        data['Unit_Price'] = unit_price_element.text.strip() 
    except Exception as e:
        data['Total_Price'] = 'N/A'
        data['Unit_Price'] = 'N/A'
        print(f"价格提取失败: {e}")

    # 5. 房源标签和交通距离
    try:
        label_container = house_item.find_element(By.CSS_SELECTOR, 'p.clearfix.label')
        #所有房源标签 (<a> 标签)
        label_elements = label_container.find_elements(By.TAG_NAME, 'a')
        labels = [a.text.strip() for a in label_elements if a.text.strip()]
        data['House_Labels'] = ' | '.join(labels)
        #单独提取地铁距离信息 (<span> 标签) 定位span.bg_none.icon_dt
        metro_span = label_container.find_element(By.CSS_SELECTOR, 'span.bg_none.icon_dt')
        data['Metro_Distance'] = metro_span.text.strip()
    except Exception as e:
        data['House_Labels'] = 'N/A'
        data['Metro_Distance'] = 'N/A'

    return data

尝试爬取一页

In [53]:
one_data = []

list_container = driver.find_element(By.CSS_SELECTOR, 'div.shop_list_4')
house_items = list_container.find_elements(By.CSS_SELECTOR, 'dl[data-bg]')

for item in house_items:
    extracted_data = extract_house_data(item) 
    one_data.append(extracted_data)

In [55]:
df_one=pd.DataFrame(one_data) 
df_one

,Title,Basic_Info_Raw,Community_Name,Detail_Address,Total_Price,Unit_Price,House_Labels,Metro_Distance
0,四开间朝南+各付各税+前面不遮挡+安静位置+诚售,3室2厅 | 146.75㎡ |低层 （共21层） | 南向 | 1998年建 |爱平,海螺花园,徐家汇 宛平南路388弄,1298万,88449元/㎡,满五,距4号线上海体育场站约726米
1,全天采光.照进每个房间.三轨交.婚房装修 业主诚意出售,3室2厅 | 166.92㎡ |高层 （共19层） | 西南向 | 2000年建 |爱平,四季园,徐家汇 东安路149弄1-8号,1580万,94656元/㎡,满五,距7号线肇嘉浜路站约463米
2,"东曼新上笋盘,正看水景,景观视野佳。有钥匙随时看。",2室2厅 | 93.83㎡ |低层 （共37层） | 南向 | 2001年建 |闵学志,东方曼哈顿,徐家汇 虹桥路168号,1080万,115101元/㎡,满五,距1号线徐家汇站约490米
3,徐家汇花园3期 2梯3户 板式房 全南2室2卫户型,2室2厅 | 126.83㎡ |中层 （共26层） | 南向 | 2004年建 |爱平,徐家汇花园,徐家汇 宛平南路255弄,1280万,100922元/㎡,满五,距7号线肇嘉浜路站约763米
4,满五WY 中间楼层 带电梯 婚房装修 内环内地段好,2室1厅 | 47.1㎡ |中层 （共6层） | 南向 | 1978年建 |爱平,东安四村,徐家汇 宛平南路727弄,478万,101486元/㎡,满五,距4号线东安路站约552米
5,零陵小区 54.29平2居 南北向精装修 安静不临街,2室1厅 | 54.29㎡ | 底层 （共6层） | 南北向 | 1986年建 |爱平,零陵小区,徐家汇 零陵小区,496万,91361元/㎡,满五,距4号线上海体育场站约219米
6,"徐家汇地铁口,满五边套全明户型,南北通风,采光俱佳",2室1厅 | 46.84㎡ |中层 （共5层） | 南向 | 1975年建 |爱平,零陵小区,徐家汇 零陵小区,384万,81981元/㎡,满五,距4号线上海体育场站约219米
7,"徐家汇商圈地铁4号线50米,南北通透,诚意出售 徐家汇",2室1厅 | 54.29㎡ |高层 （共6层） | 南北向 | 1986年建 |爱平,零陵小区,徐家汇 零陵小区,478万,88045元/㎡,满五,距4号线上海体育场站约219米
8,"新上小洋房,房东出国 ,带车位,南北通透价格好谈,来电即看",3室2厅 | 138.93㎡ |高层 （共8层） | 南北向 | 2004年建 |爱平,徐家汇花园,徐家汇 宛平南路255弄,1175万,84574元/㎡,满五,距7号线肇嘉浜路站约763米
9,徐家汇内环内 宜山路3 4 9号线 朝南无遮挡 看房方便,1室1厅 | 43.91㎡ |低层 （共24层） | 南向 | 1999年建 |闵学志,东航公寓,徐家汇 中山西路1509弄1-2号,299万,68093元/㎡,满二,距3号线虹桥路站约715米


尝试爬取多页

In [58]:
MAX_PAGES_TO_SCRAPE = 20 
current_page = 1
all_data = []

In [60]:
while current_page <= MAX_PAGES_TO_SCRAPE:
    print(f"--- 正在爬取第 {current_page} 页 ---")
    
    try:
        list_container = driver.find_element(By.CSS_SELECTOR, 'div.shop_list_4')
        house_items = list_container.find_elements(By.CSS_SELECTOR, 'dl[data-bg]')      
        
        for item in house_items:
            try:
                extracted_data = extract_house_data(item) 
                all_data.append(extracted_data)
            except Exception as e:
                print(f"提取单条房源数据时发生错误，跳过该条: {e}")
                continue
                
    except Exception as e:
        print(f"在第 {current_page} 页定位房源列表失败: {e}")
        break
        
    #翻页逻辑 
    current_page += 1 
    if current_page > MAX_PAGES_TO_SCRAPE:
        print("已爬完。")
        break 
    try:
        # "下一页" 链接，用 By.LINK_TEXT)
        next_page_link = driver.find_element(By.LINK_TEXT, '下一页')
        next_page_link.click()
        time.sleep(3)

        #随即延迟
        wait_time = random.uniform(5, 7) 
        time.sleep(wait_time) 
        
    except Exception as e:
        print(f"翻页失败，错误: {e}")
        break


--- 正在爬取第 1 页 ---
--- 正在爬取第 2 页 ---
--- 正在爬取第 3 页 ---
--- 正在爬取第 4 页 ---
--- 正在爬取第 5 页 ---
--- 正在爬取第 6 页 ---
--- 正在爬取第 7 页 ---
--- 正在爬取第 8 页 ---
--- 正在爬取第 9 页 ---
--- 正在爬取第 10 页 ---
--- 正在爬取第 11 页 ---
--- 正在爬取第 12 页 ---
--- 正在爬取第 13 页 ---
--- 正在爬取第 14 页 ---
--- 正在爬取第 15 页 ---
--- 正在爬取第 16 页 ---
--- 正在爬取第 17 页 ---
--- 正在爬取第 18 页 ---
--- 正在爬取第 19 页 ---
--- 正在爬取第 20 页 ---
已爬完。


IMPORTANT:
点击下一页会随机跳到一个不一样的类似小程序的界面，然后需要进入APP才能继续查看？？？
SOLUTION:
推测为反爬机制，找到html界面的弹窗容器<div class="downloadAPP newAppFloat oppen-xcx-list">，尝试在每次出现时关闭，方法附在下一cell。
更简单的方法是，尝试延长等待时间/随机等待，模仿真人，发现当等待时间变化时可以成功爬取。

In [62]:
final_df = pd.DataFrame(all_data)
print(f"\n批量爬取完成，共获得 {len(final_df)} 条数据。")
print(final_df.head()) 


批量爬取完成，共获得 1200 条数据。
                         Title                                Basic_Info_Raw  \
0     四开间朝南+各付各税+前面不遮挡+安静位置+诚售   3室2厅 | 146.75㎡ |低层 （共21层） | 南向 | 1998年建 |爱平   
1  全天采光.照进每个房间.三轨交.婚房装修 业主诚意出售  3室2厅 | 166.92㎡ |高层 （共19层） | 西南向 | 2000年建 |爱平   
2    东曼新上笋盘,正看水景,景观视野佳。有钥匙随时看。   2室2厅 | 93.83㎡ |低层 （共37层） | 南向 | 2001年建 |闵学志   
3    徐家汇花园3期 2梯3户 板式房 全南2室2卫户型   2室2厅 | 126.83㎡ |中层 （共26层） | 南向 | 2004年建 |爱平   
4    满五WY 中间楼层 带电梯 婚房装修 内环内地段好      2室1厅 | 47.1㎡ |中层 （共6层） | 南向 | 1978年建 |爱平   

  Community_Name   Detail_Address Total_Price Unit_Price House_Labels  \
0           海螺花园     徐家汇 宛平南路388弄       1298万   88449元/㎡           满五   
1            四季园  徐家汇 东安路149弄1-8号       1580万   94656元/㎡           满五   
2          东方曼哈顿      徐家汇 虹桥路168号       1080万  115101元/㎡           满五   
3          徐家汇花园     徐家汇 宛平南路255弄       1280万  100922元/㎡           满五   
4           东安四村     徐家汇 宛平南路727弄        478万  101486元/㎡           满五   

    Metro_Distance  
0  距4号线上海体育场站约726米  
1   距7号线肇嘉浜路站约46

In [66]:
final_df.to_excel("D:/ai finance/hw4_my_data_my_model/data_esf.xlsx")
final_df.to_parquet("D:/ai finance/hw4_my_data_my_model/data_esf.parquet", index=False)

类似地，下面爬取租房信息

In [3]:
print(plt.style.available)
%matplotlib inline
driver = webdriver.Chrome()

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


Exception managing chrome: error decoding response body


In [5]:
url_esf='https://sh.zu.fang.com/house-a019-b0103'
driver.get(url_esf)

In [21]:
def extract_rent_data(rent_item):
    data = {}
    
    # 先尝试定位根容器 dd，基于rent_item dl标签，子元素 dd 包含了所有信息
    try:
        info_container = rent_item.find_element(By.TAG_NAME, 'dd')
    except NoSuchElementException:
        print("无法找到dd容器，跳过。")
        return None
        
    # 1. 标题/小区名称：定位 p class="title" 下的 a 标签
    try:
        title_element = info_container.find_element(By.CSS_SELECTOR, 'p.title a')
        data['Title'] = title_element.get_attribute('title').strip()
    except NoSuchElementException:
        data['Title'] = 'N/A'

    # 2. 基本信息 (户型、面积、朝向、建成年份) ： p class="font15 mt12 bold"
    try:
        basic_info_element = info_container.find_element(By.CSS_SELECTOR, 'p.font15.mt12.bold')
        basic_info_raw = basic_info_element.text
        data['Basic_Info_Raw'] = basic_info_raw.replace('\n', ' | ').strip()
    except NoSuchElementException:
        data['Basic_Info_Raw'] = 'N/A'
    
    # 3. 地址 ： p class="gray6 mt12"
    try:
        address_element = info_container.find_element(By.CSS_SELECTOR, 'p.gray6.mt12')
        address_links = address_element.find_elements(By.TAG_NAME, 'a')
        data['Community_Name'] = address_links[0].text.strip() if len(address_links) > 0 else 'N/A'
        data['Detail_Address'] = address_links[1].text.strip() if len(address_links) > 1 else 'N/A'
    except NoSuchElementException:
        data['Community_Name'] = 'N/A'
        data['Detail_Address'] = 'N/A'
        
    # 4. 价格：mt5 alignC，span class="price"
    try:
        price_parent_div = info_container.find_element(By.CSS_SELECTOR, 'p.mt5.alingC')
        price_span = price_parent_div.find_element(By.CSS_SELECTOR, 'span.price')
        data['Price'] = price_span.text.strip()
        unit_raw = price_parent_div.text.replace(data['Price'], '').strip()
        data['Price_Unit'] = unit_raw.replace('\n', '').strip()       
    except NoSuchElementException:
        data['Price'] = 'N/A'
        data['Price_Unit'] = 'N/A'

    # 5. 交通距离
    try:
        distance_element = info_container.find_element(By.CSS_SELECTOR, 'span.note.subInfor')
        data['Metro_Distance'] = distance_element.text.strip()
    except NoSuchElementException:
        data['Metro_Distance'] = 'N/A'

    # 6. optional标签 ：可能有随机个
    try:
        labels_container = info_container.find_element(By.CSS_SELECTOR, 'p.mt12')        
        labels = []
        span_elements = labels_container.find_elements(By.TAG_NAME, 'span')        
        for span in span_elements:
            text = span.text.strip()
            if text:
                labels.append(text)                
        data['Labels'] = ' | '.join(labels)
    except NoSuchElementException:
        data['Labels'] = 'N/A'


    return data


调试时可先将maxpages设为2

In [32]:
MAX_PAGES_TO_SCRAPE = 20
current_page = 1
all_data = []

由于初次尝试出现定位为空的错误，此处下方优化了一些提示词

In [35]:
while current_page <= MAX_PAGES_TO_SCRAPE:
    print(f"--- 正在爬取第 {current_page} 页 ---")
    
    try:
        list_container = driver.find_element(By.CLASS_NAME, 'houseList')
        if not list_container:
            print("!!! 定位错list container")  
        rent_items = list_container.find_elements(By.CSS_SELECTOR, 'dl.list.hiddenMap.rel')   
        if not rent_items:
            print("!!! 警告：本页房源列表为空，定位失败。检查选择器。")    
        print(f"找到 {len(rent_items)} 套房源。")
        
        for item in rent_items:
            extracted_data = extract_rent_data(item)
            if extracted_data:
                all_data.append(extracted_data)
                
    except Exception as e:
        print(f"在第 {current_page} 页定位房源列表失败: {e}")
        break
        
    #翻页逻辑 
    current_page += 1 
    if current_page > MAX_PAGES_TO_SCRAPE:
        print("已爬完。")
        break 
    try:
        # "下一页" 链接，用 By.LINK_TEXT)
        next_page_link = driver.find_element(By.LINK_TEXT, '下一页')
        next_page_link.click()
        time.sleep(3)

        #随即延迟
        wait_time = random.uniform(5, 7) 
        time.sleep(wait_time) 
        
    except Exception as e:
        print(f"翻页失败，错误: {e}")
        break

--- 正在爬取第 1 页 ---
找到 60 套房源。
--- 正在爬取第 2 页 ---
找到 60 套房源。
--- 正在爬取第 3 页 ---
找到 60 套房源。
--- 正在爬取第 4 页 ---
找到 60 套房源。
--- 正在爬取第 5 页 ---
找到 60 套房源。
--- 正在爬取第 6 页 ---
找到 60 套房源。
--- 正在爬取第 7 页 ---
找到 60 套房源。
--- 正在爬取第 8 页 ---
找到 60 套房源。
--- 正在爬取第 9 页 ---
找到 60 套房源。
--- 正在爬取第 10 页 ---
找到 60 套房源。
--- 正在爬取第 11 页 ---
找到 60 套房源。
--- 正在爬取第 12 页 ---
找到 60 套房源。
--- 正在爬取第 13 页 ---
找到 60 套房源。
--- 正在爬取第 14 页 ---
找到 60 套房源。
--- 正在爬取第 15 页 ---
找到 60 套房源。
--- 正在爬取第 16 页 ---
找到 60 套房源。
--- 正在爬取第 17 页 ---
找到 60 套房源。
--- 正在爬取第 18 页 ---
找到 60 套房源。
--- 正在爬取第 19 页 ---
找到 60 套房源。
--- 正在爬取第 20 页 ---
找到 60 套房源。
已爬完。


如果这里出现了加载问题，直接修改current_page复制循环接着爬就行了

In [38]:
final_df_zu = pd.DataFrame(all_data)
print(f"\n批量爬取完成，共获得 {len(final_df_zu)} 条数据。")
print(final_df_zu.head(40)) 


批量爬取完成，共获得 1200 条数据。
                             Title    Basic_Info_Raw Community_Name  \
0      全新现代简约风格丨全屋地暖带空调丨可养宠物丨看房随时丨   整租|3室2厅|126㎡|朝南             徐汇   
1    带车位 急降2000 高区景观 南北通 徐家汇 精装拎包入  整租|3室2厅|142㎡|朝南北             徐汇   
2      徐家汇商圈 四轨交汇 欧式轻奢风 大三房两卫 全屋全配   整租|3室2厅|160㎡|朝南             徐汇   
3     全新精装修大3房,中央空调,全屋地暖,配置全品牌家具家电   整租|3室2厅|133㎡|朝南             徐汇   
4          徐家汇|宜山路|电梯一房|边套全明|干湿分离|    整租|1室1厅|37㎡|朝南             徐汇   
5            南北通户型厨卫全明,花园小区家电,繁华地段  整租|3室2厅|137㎡|朝南北             徐汇   
6   免,佣,中空,地暖,精装修,有车位,小区后排安静采光好可谈。  整租|3室2厅|199㎡|朝南北             徐汇   
7    港汇花园 徐家汇恒隆背后 带服务 会所 打扫 房租可抵税费  整租|2室2厅|145㎡|朝南北             徐汇   
8     全南户型丶婚房首租丶全新智能家具家电丶带车位丶价格可谈!   整租|4室2厅|190㎡|朝南             徐汇   
9   中央空调,全屋地暖,全屋净水,全屋智能品牌家具,带车位好停车   整租|3室2厅|157㎡|朝南             徐汇   
10  新租!淮海西路!汇宁花园!4房!仅49.8K手慢无!看房联系  整租|4室2厅|237㎡|朝南北             徐汇   
11    汇宁花园,精装修南北通4房,近地铁,近交通大學,看房方便  整租|4室2厅|235㎡|朝南北             徐汇   
12     徐家汇核心地段,汇金百货楼上,徐家汇公园全景大平层3房   整租|3室2厅|256㎡|朝南   

In [40]:
final_df_zu.to_excel("D:/ai finance/hw4_my_data_my_model/data_zu.xlsx")
final_df_zu.to_parquet("D:/ai finance/hw4_my_data_my_model/data_zu.parquet", index=False)

In [68]:
driver.quit()